In [1]:
PV = FreeAlgebra(QQ, ['v%d'%i for i in range(20)])
Ph = PolynomialRing(PV, 'h')
Pd = PolynomialRing(Ph, 'd')
d = Pd.gen()
h = Pd(Ph.gen())
zero = Pd.zero()
V = map(Pd, PV.gens())

def shifted_legendre(n):
    _d = SR.var('_d')
    _h = SR.var('_h')
    poly = (maxima.legendre_p(n,2*_d/_h-1).sage()*_h**n).expand()
    return sum(sum(QQ(ch)*h**j for ch, j in cd.coefficients())*d**i for cd, i in poly.coefficients())

def filter_high_degree(expr, n, depth=2):
    if depth == 0:
        return expr
    s = parent(expr).zero()
    v = d if depth == 2 else h
    for i, coef in enumerate(expr):
        if i > n:
            break
        s += filter_high_degree(coef, n-i, depth-1)*v**i
    return s

In [2]:
def integrate_poly(poly):
    P = parent(poly)
    g = P.gen()
    return P(sum(c*g**(i+1)*P(1/(i+1)) for i, c in enumerate(list(poly))))

def diff_poly(poly):
    P = parent(poly)
    g = P.gen()
    return P(sum(c*P(i)*g**(i-1) for i, c in enumerate(list(poly)) if i > 0))

def shift_poly(poly, n):
    P = parent(poly)
    g = P.gen()
    return P(sum(c*g**(i-n) for i, c in enumerate(list(poly)) if i >= n))

In [3]:
def calculateEta(hmax, n):

    numIt = (2*n)//3+1
    dV = sum(shifted_legendre(i)*V[i] for i in range(1, n+1))
    print dV

    def calcC(Q, R0):
        C = [[filter_high_degree(integrate_poly(Pd(Q)*Pd(1/2)), hmax)]]
        R0 = Pd(R0)
        for q in range(numIt):
            if q > 0:
                C.append([zero])
            for m in range(0, hmax//2+1):
                if q == 0:
                    R = R0 if m == 0 else zero
                else:
                    R = filter_high_degree(dV*C[q-1][m], hmax-2*m-1)
                
                C[q].append(shift_poly(
                    integrate_poly(d**m * (
                        R - diff_poly(diff_poly(C[q][m])) - filter_high_degree(C[q][m]*V[0] - V[0]*C[q][m], hmax-2*m-1)
                    )*Pd(1/2)), m+1))
        return C

    Cu = calcC(dV, zero)
    Cv = calcC(zero, dV)

    eta_u = [zero for _ in range(hmax//2+2)]
    eta_up = eta_u[:]
    eta_v = eta_u[:]
    eta_vp = eta_u[:]

    eta_u[0] = Pd.one()
    eta_up[0] = zero
    eta_v[1] = d
    eta_vp[0] = Pd.one()

    for i in range(numIt):
        for k in range(hmax//2+1):
            eta_u[k+1] += filter_high_degree(Cu[i][k] * d**(2*k+1), hmax)
            eta_v[k+1] += filter_high_degree(Cv[i][k] * d**(2*k+1), hmax)
        eta_up[0] += Cu[i][0]
        eta_vp[0] += Cv[i][0]
        for k in range(hmax//2):
            eta_up[k+1] += filter_high_degree((diff_poly(Cu[i][k]) + d*Cu[i][k+1]) * d**(2*k+1), hmax)
            eta_vp[k+1] += filter_high_degree((diff_poly(Cv[i][k]) + d*Cv[i][k+1]) * d**(2*k+1), hmax)

    return [eta_u, eta_v, eta_up, eta_vp]

In [9]:
dhmax = 8
dn = 6
hhmax = 10
hn = 8

de = calculateEta(dhmax, dn)
print "calculated eta delta"

924*v6*d^6 + (-2772*v6*h + 252*v5)*d^5 + (3150*v6*h^2 - 630*v5*h + 70*v4)*d^4 + (-1680*v6*h^3 + 560*v5*h^2 - 140*v4*h + 20*v3)*d^3 + (420*v6*h^4 - 210*v5*h^3 + 90*v4*h^2 - 30*v3*h + 6*v2)*d^2 + (-42*v6*h^5 + 30*v5*h^4 - 20*v4*h^3 + 12*v3*h^2 - 6*v2*h + 2*v1)*d + v6*h^6 - v5*h^5 + v4*h^4 - v3*h^3 + v2*h^2 - v1*h
calculated eta delta


In [10]:
# 2 minuten
he = calculateEta(hhmax, hn)
print "calculated eta h"

12870*v8*d^8 + (-51480*v8*h + 3432*v7)*d^7 + (84084*v8*h^2 - 12012*v7*h + 924*v6)*d^6 + (-72072*v8*h^3 + 16632*v7*h^2 - 2772*v6*h + 252*v5)*d^5 + (34650*v8*h^4 - 11550*v7*h^3 + 3150*v6*h^2 - 630*v5*h + 70*v4)*d^4 + (-9240*v8*h^5 + 4200*v7*h^4 - 1680*v6*h^3 + 560*v5*h^2 - 140*v4*h + 20*v3)*d^3 + (1260*v8*h^6 - 756*v7*h^5 + 420*v6*h^4 - 210*v5*h^3 + 90*v4*h^2 - 30*v3*h + 6*v2)*d^2 + (-72*v8*h^7 + 56*v7*h^6 - 42*v6*h^5 + 30*v5*h^4 - 20*v4*h^3 + 12*v3*h^2 - 6*v2*h + 2*v1)*d + v8*h^8 - v7*h^7 + v6*h^6 - v5*h^5 + v4*h^4 - v3*h^3 + v2*h^2 - v1*h
calculated eta h


In [11]:
header = """\
#include "./matscs_formulas.h"

template<typename Scalar>
void calculate_tcoeff_matrix(
        int n,
        Scalar h,
        const std::array<Eigen::Matrix<Scalar, -1, -1>, MATSCS_N> &vs,
        Eigen::Array<Eigen::Matrix<Scalar, -1, -1>, MATSCS_ETA_delta, MATSCS_HMAX_delta> &tDelta,
        std::array<Eigen::Matrix<Scalar, -1, -1>, MATSCS_ETA_h> &tH) {
        typedef Eigen::Matrix<Scalar, -1, -1> MatrixXs;
"""

for i in range(0, max(hn, dn)+1):
    header += "    const MatrixXs &v%d = vs[%d];\n"%(i, i)

header += "    const Scalar &h1 = h;\n"
for i in range(2, max(hhmax, dhmax)+1):
    header += "    const Scalar h%d = h*h%d;\n"%(i, i-1)

footer = """\
};

#define INSTANTIATE_MATSLISE(Scalar) \\
template void calculate_tcoeff_matrix<Scalar>(int, Scalar h, const std::array<Eigen::Matrix<Scalar, -1, -1>, MATSCS_N> &, Eigen::Array<Eigen::Matrix<Scalar, -1, -1>, MATSCS_ETA_delta, MATSCS_HMAX_delta> &, std::array<Eigen::Matrix<Scalar, -1, -1>, MATSCS_ETA_h> &);

#include "../util/instantiate.h"
"""
    
body = ""

def expr_to_C(expr, depth=2, lhs=None):
    if depth == 0:
        return str(float(expr.n()))
        return '(D('+str(expr.numerator())+'L)/D('+str(expr.denominator())+'L))'
    if depth == 1:
        vs = PV.gens()
        r = []
        for k, v in dict(expr).items():
            if k == PV(1):
                if v == 1:
                    r.append('MatrixXs::Identity(n, n)')
                else:
                    r.append(expr_to_C(v, depth-1) + ' * MatrixXs::Identity(n, n)')
            else:
                r.append(expr_to_C(v, depth-1) + "".join(
                    ('*' + str(gen))*m for gen, m in list(k) if m > 0
                ))
        if len(r) == 0:
            return 'MatrixXs::Zero(n,n)'
        return '+'.join(r)

    r = list(expr)
    if lhs:
        if len(r) == 0:
            return '    ' + lhs + ' = MatrixXs::Zero(n,n); '
        elif len(r) == 1:
            return '    ' + lhs + ' = ' + expr_to_C(r[0], depth-1)+';'
        else:
            l = ""
            first = True
            for i, v in enumerate(r):
                if v != 0:
                    l += '    ' + lhs+(" = " if first else " += ")+('h%d * '%i if i > 0 else '') + "(" + expr_to_C(v, depth-1) + ");\n"
                    first = False;
            return l
        
    else:
        if len(r) == 0:
            return 'MatrixXs::Zero(n,n)'
        elif len(r) == 1:
            return expr_to_C(r[0], depth-1)
        else:
            l = []
            for i, v in enumerate(r):
                if v != 0:
                    l.append(('h%d * '%i if i > 0 else '') + "(" + expr_to_C(v, depth-1) + ")")
            return ' + '.join(l)

for j in range(1+hhmax//2):
    if j <= dhmax/2:
        exprs = [filter_high_degree(de[k][j], dhmax) for k in range(4)]
        for i in range(dhmax+1):
            if all(e[i] == Pd(0) for e in exprs):
                body += "    tDelta(%d, %d) = MatrixXs::Zero(2*n, 2*n);\n"%(j, i)
            else:
                body += "    tDelta(%d, %d).resize(2*n, 2*n);\n"%(j, i)
                for e, key in zip(exprs, ["0, 0", "0, n", "n, 0", "n, n"]):
                    body += "    tDelta(%d, %d).block(%s, n, n) = %s;\n"%(j, i, key, expr_to_C(e[i]))
    h_exprs = [filter_high_degree(he[k][j], hhmax) for k in range(4)]
    body += "    tH[%d].resize(2*n, 2*n);\n"%(j)
    for e, key in zip(h_exprs, ["0, 0", "0, n", "n, 0", "n, n"]):
        body += expr_to_C(Ph(e(h)), lhs="tH[%d].block(%s, n, n)"%(j,key))
        

In [12]:
from collections import defaultdict
import re
variable = "([a-z][a-zA-Z0-9_]*)"
pattern = re.compile("\\b"+variable+"\\s*\\*\\s*"+variable+"\\b")

head = []
replaced_body = body
replace_header = ""
while True:
    var_freqs = defaultdict(int)
    for v in pattern.findall(replaced_body):
        var_freqs[v] += 1
    if len(var_freqs) == 0:
        break
    m = max(var_freqs.keys(), key=lambda k: var_freqs[k])
    count = var_freqs[m]
    # print m
    if count < 2:
        break
    v1, v2 = m
    name = v1+"_"+v2
    replaced_body = re.sub("\\b"+v1+"\\s*\\*\\s*"+v2+"\\b", name, replaced_body)
    replace_header += '    const MatrixXs ' + name+ ' = '+v1+' * '+v2+';\n'

In [13]:
with open('matscs_formulas.cpp', 'w') as f:
    f.write(header + replace_header + replaced_body+footer)